<a href="https://colab.research.google.com/github/Iallen520/lhy_DL_Hw/blob/master/hw4_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>连接</a>

根据这个改进：https://www.cnblogs.com/dogecheng/p/11565530.html 标点符号 停用词

1）Word2Vec使用了无标签数据集导致词表过大

2）训练10个epoch 测试集没提升，训练集到acc到90%——>过拟合 （训练集19万和18万没区别）

3）网络（加多一个全连接），测试集没有提升——>数据的问题

4）原来的数据can ' t存在空格，所以用txt的替换功能

In [1]:
import os
os.getcwd()

'E:\\jupyter item\\LHY_RNN'

# Recurrent Neural Networks

本次作業是要讓同學接觸NLP當中一個簡單的task——句子分類(文本分類)

給定一個句子，判斷他有沒有惡意(負面標1，正面標0)

若有任何問題，歡迎來信至助教信箱ntu-ml-2020spring-ta@googlegroups.com

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
# path_prefix = 'drive/My Drive/Colab Notebooks/hw4 - Recurrent Neural Network'
path_prefix = './'

### Download Dataset
有三個檔案，分別是 training_label.txt、training_nolabel.txt、testing_data.txt

- training_label.txt：有 label 的 training data（句子配上 0 or 1，+++$+++ 只是分隔符號，不要理它）
    - e.g., 1 +++$+++ are wtf ... awww thanks !

- training_nolabel.txt：沒有 label 的 training data（只有句子），用來做 semi-supervised learning
    - ex: hates being this burnt !! ouch

- testing_data.txt：你要判斷 testing data 裡面的句子是 0 or 1

    >id,text

    >0,my dog ate our dinner . no , seriously ... he ate it .

    >1,omg last day sooon n of primary noooooo x im gona be swimming out of school wif the amount of tears am gona cry

    >2,stupid boys .. they ' re so .. stupid !

In [0]:
# !wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1dPHIl8ZnfDz_fxNd2ZeBYedTat2lfxcO' -O 'drive/My Drive/Colab Notebooks/hw8-RNN/data/training_label.txt'
# !wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1x1rJOX_ETqnOZjdMAbEE2pqIjRNa8xcc' -O 'drive/My Drive/Colab Notebooks/hw8-RNN/data/training_nolabel.txt'
# !wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=16CtnQwSDCob9xmm6EdHHR7PNFNiOrQ30' -O 'drive/My Drive/Colab Notebooks/hw8-RNN/data/testing_data.txt'

!gdown --id '1lz0Wtwxsh5YCPdqQ3E3l_nbfJT1N13V8' --output data.zip #Google Drive直链工具
!unzip data.zip
!ls

# !ls 'drive/My Drive/Colab Notebooks/hw4 - Recurrent Neural Network/data'

Downloading...
From: https://drive.google.com/uc?id=1lz0Wtwxsh5YCPdqQ3E3l_nbfJT1N13V8
To: /content/data.zip
45.1MB [00:00, 203MB/s]
Archive:  data.zip
  inflating: training_label.txt      
  inflating: testing_data.txt        
  inflating: training_nolabel.txt    
data.zip     testing_data.txt	 training_nolabel.txt
sample_data  training_label.txt


In [3]:
# this is for filtering the warnings
import warnings
warnings.filterwarnings('ignore')

# 怎么读取数据的

In [14]:
#把testing時需要的data讀進來
import re
from sklearn.feature_extraction import text
stop = '[‘’"#$%&\()*+,./:;<=>@[\\]^_`{|}~]+'
# with open('./test.txt', 'r') as f:
#     lines = f.readlines()
# #     print(lines)  
#     X = ["".join(line.strip('\n').split(",")[1:]).strip() for line in lines[1:]]#split()如果句子有多个逗号就会分成几句。需要join()来连接
#     print(X)
#     X = [re.sub(stop, "", sentence).strip() for sentence in X]
#     X = [sen.split() for sen in X]
#     print(X)
stop_words = set(text.ENGLISH_STOP_WORDS)
# print(stop_words)
line = []
line1 = []
with open('./train.txt', 'r', encoding='UTF-8') as f: #需要encoding='UTF-8'
    lines = f.readlines()
    X = [re.sub(stop, "", sentence).strip() for sentence in lines]
    print(X)
#     lines = [line.strip('\n').split() for line in X]
    for sentence in X:
        label = sentence[0]
        tmp = [word for word in sentence[1:].strip('\n').split() if len(word)>1 or word in ['i','!','?']  ] # if word not in stop_words
        line.append(tmp)
        line1.append(label)
x = line
y = line1

#     lines = f.readlines()
#     lines = [line.strip('\n').split(' ') for line in lines]
# x = [line[2:] for line in lines]
# y = [line[0] for line in lines]
print(x)
print(y)

["0  wish weekend  but not really also  cuz next monday is exam and i haven't studied at all yet hate exam  grr", "1  check this vid out  you'll piss yourself laughin", '0  damn you gavin !!!!!! i want my computer back !!!!', '0  got a bloody wheel clamp yesterday ï ¿ ½150 for 15 mins parking', "0  homework and summer school  we'll go soon though !"]
[['wish', 'weekend', 'but', 'not', 'really', 'also', 'cuz', 'next', 'monday', 'is', 'exam', 'and', 'i', "haven't", 'studied', 'at', 'all', 'yet', 'hate', 'exam', 'grr'], ['check', 'this', 'vid', 'out', "you'll", 'piss', 'yourself', 'laughin'], ['damn', 'you', 'gavin', '!!!!!!', 'i', 'want', 'my', 'computer', 'back', '!!!!'], ['got', 'bloody', 'wheel', 'clamp', 'yesterday', '½150', 'for', '15', 'mins', 'parking'], ['homework', 'and', 'summer', 'school', "we'll", 'go', 'soon', 'though', '!']]
['0', '1', '0', '0', '0']


## 数据预处理

In [2]:
from string import punctuation
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [19]:
import numpy as np
np.random.seed(111)
lines = [
    [1,1,1],
    [2,2,2],
    [3,3,3],
    [4,4,4],
]
lines1 = np.array(lines)
lines2 = np.array(lines)
np.random.shuffle(lines1)
index = np.random.permutation(len(lines2))
print(lines1)
print(index)
print(lines2[index])

[[2 2 2]
 [3 3 3]
 [4 4 4]
 [1 1 1]]
[3 1 2 0]
[[4 4 4]
 [2 2 2]
 [3 3 3]
 [1 1 1]]


### Utils

In [5]:
# utils.py
# 這個block用來先定義一些等等常用到的函式
import torch
import numpy as np
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
import re
stop = '[’"#$%&\()*+,./:;<=>@[\\]^_`{|}~]+'
def load_training_data(path='training_label.txt'):
    # 把training時需要的data讀進來
    # 如果是'training_label.txt'，需要讀取label，如果是'training_nolabel.txt'，不需要讀取label
    global stop
    line=[]
    line1=[]
    if 'training_label' in path:
        with open(path, 'r', encoding='UTF-8') as f: #需要encoding='UTF-8'
#             lines = f.readlines()
#             X = [re.sub(stop, "", sentence).strip() for sentence in lines]
#             lines = [line.strip('\n').split() for line in X]
#         x = [line[1:] for line in lines]
#         y = [line[0] for line in lines]
            lines = f.readlines()
            X = [re.sub(stop, "", sentence).strip() for sentence in lines]
            for sentence in X:
                label = sentence[0]
                tmp = [word for word in sentence[1:].strip('\n').split() if len(word)>1 or word in ['i','!','?']]
                line.append(tmp)
                line1.append(label)
        print(len(line1))
        return line, line1
    else:
        with open(path, 'r', encoding='UTF-8') as f:
            lines = f.readlines()
            X = [re.sub(stop, "", sentence).strip() for sentence in lines]
            lines = [line.strip('\n').split() for line in X]
        return lines

def load_testing_data(path='testing_data'):
    # 把testing時需要的data讀進來
    global stop
    line=[]
    with open(path, 'r', encoding='UTF-8') as f:
        lines = f.readlines()
        X = ["".join(line.strip('\n').split(",")[1:]).strip() for line in lines[1:]]
        X = [re.sub(stop, "", sentence).strip() for sentence in X]
#         X = [sen.split() for sen in X]
        for sentence in X:
            tmp = [word for word in sentence.strip('\n').split() if len(word)>1 or word in ['i','!','?']]
            line.append(tmp)
        print(len(line))
    return line

def evaluation(outputs, labels):
    #outputs => probability (float)
    #labels => labels
    outputs[outputs>=0.5] = 1 # 大於等於0.5為有惡意
    outputs[outputs<0.5] = 0 # 小於0.5為無惡意
    correct = torch.sum(torch.eq(outputs, labels)).item()
    return correct

### Train Word to Vector

In [15]:
# w2v.py
# 這個block是用來訓練word to vector 的 word embedding
# 注意！這個block在訓練word to vector時是用cpu，可能要花到10分鐘以上
import os
import numpy as np
import pandas as pd
import argparse
from gensim.models import word2vec

def train_word2vec(x):
    # 訓練word to vector 的 word embedding
    model = word2vec.Word2Vec(x, size=250, window=5, min_count=5, workers=12, iter=10, sg=1) #sg0：CBOW sg1：skip
    return model

if __name__ == "__main__":
    print("loading training data ...")
    train_x, y = load_training_data('./hw4/training_label.txt')
#     train_x_no_label = load_training_data('./hw4/training_nolabel.txt')

    print("loading testing data ...")
    test_x = load_testing_data('./hw4/testing_data.txt')

    model = train_word2vec(train_x  + test_x) #+ train_x_no_label
    
    print("saving model ...")
    # model.save(os.path.join(path_prefix, 'model/w2v_all.model'))
    model.save(os.path.join(path_prefix, 'w2v_all.model'))

loading training data ...
200000
loading testing data ...
200000
saving model ...


### Data Preprocess

In [17]:
# preprocess.py
# 這個block用來做data的預處理
from torch import nn
from gensim.models import Word2Vec

class Preprocess():
    def __init__(self, sentences, sen_len, w2v_path="./w2v.model"):
        self.w2v_path = w2v_path
        self.sentences = sentences
        self.sen_len = sen_len
        self.idx2word = []
        self.word2idx = {}
        self.embedding_matrix = []
    def get_w2v_model(self):
        # 把之前訓練好的word to vec 模型讀進來
        self.embedding = Word2Vec.load(self.w2v_path)
        self.embedding_dim = self.embedding.vector_size
    def add_embedding(self, word):
        # 把word加進embedding，並賦予他一個隨機生成的representation vector
        # word只會是"<PAD>"或"<UNK>"
        vector = torch.empty(1, self.embedding_dim)
        torch.nn.init.uniform_(vector)
        self.word2idx[word] = len(self.word2idx)
        self.idx2word.append(word)
        self.embedding_matrix = torch.cat([self.embedding_matrix, vector], 0)
    def make_embedding(self, load=True):
        print("Get embedding ...")
        # 取得訓練好的 Word2vec word embedding
        if load:
            print("loading word to vec model ...")
            self.get_w2v_model()
        else:
            raise NotImplementedError
        # 製作一個 word2idx 的 dictionary
        # 製作一個 idx2word 的 list
        # 製作一個 word2vector 的 list
        for i, word in enumerate(self.embedding.wv.vocab):
            print('get words #{}'.format(i+1), end='\r') #\r表示回到行首
            #e.g. self.word2index['魯'] = 1 
            #e.g. self.index2word[1] = '魯'
            #e.g. self.vectors[1] = '魯' vector
            self.word2idx[word] = len(self.word2idx)
            self.idx2word.append(word)
            self.embedding_matrix.append(self.embedding[word])
        print('')
        self.embedding_matrix = torch.tensor(self.embedding_matrix)
        # 將"<PAD>"跟"<UNK>"加進embedding裡面
        self.add_embedding("<PAD>")
        self.add_embedding("<UNK>")
        print("total words: {}".format(len(self.embedding_matrix)))
        return self.embedding_matrix
    def pad_sequence(self, sentence):
        # 將每個句子變成一樣的長度
        if len(sentence) > self.sen_len:
            sentence = sentence[:self.sen_len]
        else:
            pad_len = self.sen_len - len(sentence)
            for _ in range(pad_len):
                sentence.append(self.word2idx["<PAD>"])
        assert len(sentence) == self.sen_len
        return sentence
    def sentence_word2idx(self):
        # 把句子裡面的字轉成相對應的index
        sentence_list = []
        for i, sen in enumerate(self.sentences):
            print('sentence count #{}'.format(i+1), end='\r')
            sentence_idx = []
            for word in sen:
                if (word in self.word2idx.keys()):
                    sentence_idx.append(self.word2idx[word])
                else:
                    sentence_idx.append(self.word2idx["<UNK>"])
            # 將每個句子變成一樣的長度
            sentence_idx = self.pad_sequence(sentence_idx)
            sentence_list.append(sentence_idx)
        return torch.LongTensor(sentence_list)
    def labels_to_tensor(self, y):
        # 把labels轉成tensor
        y = [int(label) for label in y] #int()说明y是一维
        return torch.LongTensor(y)


### Dataset

In [18]:
# data.py
# 實作了dataset所需要的'__init__', '__getitem__', '__len__'
# 好讓dataloader能使用
import torch
from torch.utils import data

class TwitterDataset(data.Dataset):
    """
    Expected data shape like:(data_num, data_len)
    Data can be a list of numpy array or a list of lists
    input data shape : (data_num, seq_len, feature_dim)
    
    __len__ will return the number of data
    """
    def __init__(self, X, y):
        self.data = X
        self.label = y
    def __getitem__(self, idx):
        if self.label is None: return self.data[idx]
        return self.data[idx], self.label[idx]
    def __len__(self):
        return len(self.data)

# nn.Embedding

所以Embedding会增加一个维度，即feature_dim

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

word2idx = {'hello':0, 'world':1, 'goodbye':2}
embeds = nn.Embedding(3,5)
hello_idx = torch.LongTensor([word2idx['hello']])
hello_idx = Variable(hello_idx)
hello_embed = embeds(hello_idx)
print(hello_embed)
random_idx = torch.LongTensor([0,2,1,2])
random_idx = Variable(random_idx)
random_embed = embeds(random_idx)
print(random_embed)

tensor([[-0.2278, -1.5511, -0.1098,  0.7454,  1.2229]],
       grad_fn=<EmbeddingBackward>)
tensor([[-0.2278, -1.5511, -0.1098,  0.7454,  1.2229],
        [ 1.7156,  1.3619,  0.2018, -1.0366,  1.3712],
        [-0.7197,  1.7748,  0.7382,  1.1914,  1.0706],
        [ 1.7156,  1.3619,  0.2018, -1.0366,  1.3712]],
       grad_fn=<EmbeddingBackward>)


# gensim预训练模型

In [16]:
from gensim.models import Word2Vec
embedding = Word2Vec.load('./w2v_all.model')
embedding_dim = embedding.vector_size
print(embedding_dim)
print(len(embedding.wv.vocab))
print(embedding.wv.vocab['can'])
print(embedding.wv['?'][:10])
print(embedding['?'][0:10])
print(embedding.wv.vectors.shape)

250
24711
Vocab(count:17820, index:37, sample_int:3475032967)
[ 0.1594226   0.15222673  0.11124863 -0.1402431   0.13791953 -0.20665413
 -0.19474867  0.23798528  0.10662556  0.1699507 ]
[ 0.1594226   0.15222673  0.11124863 -0.1402431   0.13791953 -0.20665413
 -0.19474867  0.23798528  0.10662556  0.1699507 ]
(24711, 250)


## Model

version0 : num_layers=1 dropout=0.5

version1 : num_layers=2 dropout=0.3

version2 : num_layers=2 dropout=0.5 add Linear（hidden->50,50->1）

In [19]:
# model.py
# 這個block是要拿來訓練的模型
#embedding_dim=250, hidden_dim=250, num_layers=1
import torch
from torch import nn
class LSTM_Net(nn.Module):
    def __init__(self, embedding, embedding_dim, hidden_dim, num_layers, dropout=0.5, fix_embedding=True):
        super(LSTM_Net, self).__init__()
        # 製作 embedding layer
        self.embedding = torch.nn.Embedding(embedding.size(0),embedding.size(1))
        self.embedding.weight = torch.nn.Parameter(embedding)
        # 是否將 embedding fix住，如果fix_embedding為False，在訓練過程中，embedding也會跟著被訓練
        self.embedding.weight.requires_grad = False if fix_embedding else True
        self.embedding_dim = embedding.size(1)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True,  bidirectional=True)
        self.classifier = nn.Sequential( nn.Dropout(dropout),
                                         nn.Linear(500, 1),
                                         nn.Sigmoid() )
    def forward(self, inputs):
        inputs = self.embedding(inputs)
        x, _ = self.lstm(inputs, None)
        # x 的 dimension (batch, seq_len, hidden_size)
        # 取用 LSTM 最後一層的 hidden state
        x = x[:, -1, :] 
        x = self.classifier(x)
        return x

### Train

In [20]:
# train.py
# 這個block是用來訓練模型的
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

def training(batch_size, n_epoch, lr, model_dir, train, valid, model, device):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('\nstart training, parameter total:{}, trainable:{}\n'.format(total, trainable))
    model.train() # 將model的模式設為train，這樣optimizer就可以更新model的參數
    criterion = nn.BCELoss() # 定義損失函數，這裡我們使用binary cross entropy loss
    t_batch = len(train)
    
    print('t_batch: {} \n'.format(t_batch))
    
    v_batch = len(valid) 
    optimizer = optim.Adam(model.parameters(), lr=lr) # 將模型的參數給optimizer，並給予適當的learning rate
    total_loss, total_acc, best_acc = 0, 0, 0
    for epoch in range(n_epoch):
        total_loss, total_acc = 0, 0
        # 這段做training
        for i, (inputs, labels) in enumerate(train):
            inputs = inputs.to(device, dtype=torch.long) # device為"cuda"，將inputs轉成torch.cuda.LongTensor
            labels = labels.to(device, dtype=torch.float) # device為"cuda"，將labels轉成torch.cuda.FloatTensor，因為等等要餵進criterion，所以型態要是float
            optimizer.zero_grad() # 由於loss.backward()的gradient會累加，所以每次餵完一個batch後需要歸零
            outputs = model(inputs) # 將input餵給模型
            
            #input的shape： (batch,seq_len)
            #output的shape： (batch, hidden_size = 1)
            
            outputs = outputs.squeeze() # 去掉最外面的dimension，好讓outputs可以餵進criterion()
            loss = criterion(outputs, labels) # 計算此時模型的training loss
            loss.backward() # 算loss的gradient
            optimizer.step() # 更新訓練模型的參數
            
            #evaluation在utils
            correct = evaluation(outputs, labels) # 計算此時模型的training accuracy
            total_acc += (correct / batch_size)
            total_loss += loss.item()
            print('[ Epoch{}: {}/{} ] loss:{:.3f} acc:{:.3f} '.format(
                epoch+1, i+1, t_batch, loss.item(), correct*100/batch_size), end='\r')
        print('\nTrain | Loss:{:.5f} Acc: {:.3f}'.format(total_loss/t_batch, total_acc/t_batch*100))

        # 這段做validation
        model.eval() # 將model的模式設為eval，這樣model的參數就會固定住
        with torch.no_grad():
            total_loss, total_acc = 0, 0
            for i, (inputs, labels) in enumerate(valid):
                inputs = inputs.to(device, dtype=torch.long) # device為"cuda"，將inputs轉成torch.cuda.LongTensor
                labels = labels.to(device, dtype=torch.float) # device為"cuda"，將labels轉成torch.cuda.FloatTensor，因為等等要餵進criterion，所以型態要是float
                outputs = model(inputs) # 將input餵給模型
                outputs = outputs.squeeze() # 去掉最外面的dimension，好讓outputs可以餵進criterion()
                loss = criterion(outputs, labels) # 計算此時模型的validation loss
                correct = evaluation(outputs, labels) # 計算此時模型的validation accuracy
                total_acc += (correct / batch_size)
                total_loss += loss.item()

            print("Valid | Loss:{:.5f} Acc: {:.3f} ".format(total_loss/v_batch, total_acc/v_batch*100))
            if total_acc > best_acc:
                # 如果validation的結果優於之前所有的結果，就把當下的模型存下來以備之後做預測時使用
                best_acc = total_acc
                #torch.save(model, "{}/val_acc_{:.3f}.model".format(model_dir,total_acc/v_batch*100))
                torch.save(model, "{}/version1_ckpt.model".format(model_dir))
                print('saving model with acc {:.3f}'.format(total_acc/v_batch*100))
        print('-----------------------------------------------')
        model.train() # 將model的模式設為train，這樣optimizer就可以更新model的參數（因為剛剛轉成eval模式）

### Test

In [0]:
# test.py
# 這個block用來對testing_data.txt做預測
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

def testing(batch_size, test_loader, model, device):
    model.eval()
    ret_output = []
    with torch.no_grad():
        for i, inputs in enumerate(test_loader):
            inputs = inputs.to(device, dtype=torch.long)
            outputs = model(inputs)
            outputs = outputs.squeeze()
            outputs[outputs>=0.5] = 1 # 大於等於0.5為負面
            outputs[outputs<0.5] = 0 # 小於0.5為正面
            ret_output += outputs.int().tolist()
    
    return ret_output

### Main

In [21]:
# main.py
import os
import torch
import argparse
import numpy as np
from torch import nn
from gensim.models import word2vec
from sklearn.model_selection import train_test_split

# 通過torch.cuda.is_available()的回傳值進行判斷是否有使用GPU的環境，如果有的話device就設為"cuda"，沒有的話就設為"cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 處理好各個data的路徑
train_with_label = os.path.join(path_prefix, 'hw4/training_label.txt')
# train_no_label = os.path.join(path_prefix, 'training_nolabel.txt')
testing_data = os.path.join(path_prefix, 'hw4/testing_data.txt')

w2v_path = os.path.join(path_prefix, 'w2v_all.model') # 處理word to vec model的路徑

# 定義句子長度、要不要固定embedding、batch大小、要訓練幾個epoch、learning rate的值、model的資料夾路徑
sen_len = 30
fix_embedding = True # fix embedding during training
batch_size = 128
epoch = 7
lr = 0.001
# model_dir = os.path.join(path_prefix, 'model/') # model directory for checkpoint model
model_dir = './hw4/logs/' #path_prefix # model directory for checkpoint model

print("loading data ...") # 把'training_label.txt'跟'training_nolabel.txt'讀進來
train_x, y = load_training_data(train_with_label)
# train_x_no_label = load_training_data(train_no_label)

# 對input跟labels做預處理
preprocess = Preprocess(train_x, sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True) #self.embedding_matrix
train_x = preprocess.sentence_word2idx()
y = preprocess.labels_to_tensor(y)

# 製作一個model的對象
model = LSTM_Net(embedding, embedding_dim=250, hidden_dim=250, num_layers=2, dropout=0.5,fix_embedding=fix_embedding)
model = model.to(device) # device為"cuda"，model使用GPU來訓練(餵進去的inputs也需要是cuda tensor)

# 把data分為training data跟validation data(將一部份training data拿去當作validation data)
X_train, X_val, y_train, y_val = train_x[:190000], train_x[190000:], y[:190000], y[190000:]

# 把data做成dataset供dataloader取用
train_dataset = TwitterDataset(X=X_train, y=y_train)
val_dataset = TwitterDataset(X=X_val, y=y_val)

# 把data 轉成 batch of tensors
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = batch_size,
                                            shuffle = True,
                                            ) #num_workers = 8

val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                                            batch_size = batch_size,
                                            shuffle = False,
                                            ) #num_workers = 8

# 開始訓練
training(batch_size, epoch, lr, model_dir, train_loader, val_loader, model, device)

loading data ...
200000
Get embedding ...
loading word to vec model ...
get words #24711
total words: 24713
sentence count #200000
start training, parameter total:8686751, trainable:2508501

t_batch: 1485 

[ Epoch1: 1485/1485 ] loss:0.467 acc:29.688 
Train | Loss:0.48580 Acc: 75.561
Valid | Loss:0.44565 Acc: 77.690 
saving model with acc 77.690
-----------------------------------------------
[ Epoch2: 1485/1485 ] loss:0.390 acc:29.688 
Train | Loss:0.41803 Acc: 80.735
Valid | Loss:0.42092 Acc: 79.480 
saving model with acc 79.480
-----------------------------------------------
[ Epoch3: 1485/1485 ] loss:0.340 acc:30.469 
Train | Loss:0.39915 Acc: 81.729
Valid | Loss:0.41202 Acc: 80.498 
saving model with acc 80.498
-----------------------------------------------
[ Epoch4: 1485/1485 ] loss:0.297 acc:31.250 
Train | Loss:0.38159 Acc: 82.743
Valid | Loss:0.41244 Acc: 80.390 
-----------------------------------------------
[ Epoch5: 1485/1485 ] loss:0.420 acc:29.688 
Train | Loss:0.36126 

KeyboardInterrupt: 

### Predict and Write to csv file

In [0]:
# 開始測試模型並做預測
print("loading testing data ...")
test_x = load_testing_data(testing_data)
preprocess = Preprocess(test_x, sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
test_x = preprocess.sentence_word2idx()
test_dataset = TwitterDataset(X=test_x, y=None)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                            batch_size = batch_size,
                                            shuffle = False,
                                            ) #num_workers = 8
print('\nload model ...')
model = torch.load(os.path.join('./hw4/logs', 'ckpt.model'))
outputs = testing(batch_size, test_loader, model, device)

# 寫到csv檔案供上傳kaggle
tmp = pd.DataFrame({"id":[str(i) for i in range(len(test_x))],"label":outputs})
print("save csv ...")
tmp.to_csv(os.path.join(path_prefix, 'predict.csv'), index=False)
print("Finish Predicting")

# 以下是使用command line上傳到kaggle的方式
# 需要先pip install kaggle、Create API Token，詳細請看https://github.com/Kaggle/kaggle-api以及https://www.kaggle.com/code1110/how-to-submit-from-google-colab
# kaggle competitions submit [competition-name] -f [csv file path]] -m [message]
# ex: kaggle competitions submit ml-2020spring-hw4 -f output/predict.csv -m "......"

loading testing data ...
Get embedding ...
loading word to vec model ...


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


get words #1648

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


get words #55777
total words: 55779
sentence count #200000
load model ...
save csv ...
Finish Predicting


#### Check where the files are

In [0]:
!pwd
!ls

/content
ckpt.model   testing_data.txt	   w2v_all.model.trainables.syn1neg.npy
data.zip     training_label.txt    w2v_all.model.wv.vectors.npy
predict.csv  training_nolabel.txt
sample_data  w2v_all.model


### Run 20 epochs on n98
real	3m33.317s

user	3m29.813s

sys	1m9.469s